In [2]:
from tqdm import tqdm
import json
import requests
import geopandas as gpd
import geocoder
import pandas as pd
import numpy as np
from shapely.geometry import LineString, Point, Polygon
from func import *

In [3]:
specified_order = [
    '北投區', '士林區', '內湖區', '南港區',
    '松山區', '信義區', '中山區', '大同區',
    '中正區', '萬華區', '大安區', '文山區'
]

In [4]:
population_original = get_datataipei_api('c8f5b53d-ef3d-4321-ae8e-58cd2a5ee73c')
population = population_original.copy()
population

,_id,_importdate,年份,月份,區域代碼,區域別,性別,總計,0歲數量,1歲數量,...,91歲數量,92歲數量,93歲數量,94歲數量,95歲數量,96歲數量,97歲數量,98歲數量,99歲數量,100歲以上
0,1,"{'date': '2023-11-14 13:55:11.563907', 'timezo...",112,1,63000000,總計,計,2488043,13244,16616,...,4239,3564,2999,2363,1888,1460,1016,735,512,1072
1,2,"{'date': '2023-11-14 13:55:11.577019', 'timezo...",112,1,63000000,總計,男,1180304,6784,8539,...,1817,1556,1323,1047,851,638,478,319,211,531
2,3,"{'date': '2023-11-14 13:55:11.579146', 'timezo...",112,1,63000000,總計,女,1307739,6460,8077,...,2422,2008,1676,1316,1037,822,538,416,301,541
3,4,"{'date': '2023-11-14 13:55:11.581200', 'timezo...",112,1,63000010,松山區,計,190772,938,1267,...,397,292,267,202,213,138,94,71,55,127
4,5,"{'date': '2023-11-14 13:55:11.582943', 'timezo...",112,1,63000010,松山區,男,89059,482,654,...,160,123,118,74,95,68,46,32,24,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14065,14066,"{'date': '2023-11-14 13:55:37.997899', 'timezo...",112,10,63000120041,大屯里,男,643,3,5,...,0,1,2,0,1,1,0,0,0,0
14066,14067,"{'date': '2023-11-14 13:55:37.999587', 'timezo...",112,10,63000120041,大屯里,女,561,4,4,...,3,0,2,1,1,0,4,1,0,0
14067,14068,"{'date': '2023-11-14 13:55:38.001177', 'timezo...",112,10,63000120042,湖田里,計,862,5,4,...,1,2,3,3,1,0,1,0,0,1
14068,14069,"{'date': '2023-11-14 13:55:38.002735', 'timezo...",112,10,63000120042,湖田里,男,472,1,1,...,1,2,1,1,0,0,1,0,0,1


In [8]:
population_total = population[population["區域別"] == "總計"]
population_total = population_total[population_total["性別"] == "計"]
# Specify the columns you want to drop in a list
columns_to_drop = ["_id", "_importdate", "區域代碼", "區域別", "總計", "100歲以上", "性別"]
columns_to_drop += [f"{i}歲數量" for i in range(6, 100)]
# Use the drop method to remove the specified columns
population_total.drop(columns=columns_to_drop, inplace=True)
population_total

,年份,月份,0歲數量,1歲數量,2歲數量,3歲數量,4歲數量,5歲數量
0,112,1,13244,16616,17363,19406,20452,21942
1407,112,2,13267,16376,17210,19361,20273,21588
2814,112,3,13455,16018,17101,19258,20195,21348
4221,112,4,13463,15762,16936,19087,20047,21345
5628,112,5,13724,15624,16809,18863,19954,21171
7035,112,6,13798,15469,16831,18795,19810,21137
8442,112,7,13878,15416,16762,18635,19882,21130
9849,112,8,14009,15348,16735,18402,19683,21022
11256,112,9,14040,15110,16797,18119,19646,20838
12663,112,10,14355,14875,16741,17935,19641,20578


In [10]:
population_total.reset_index(drop=True, inplace=True)
population_total

,年份,月份,0歲數量,1歲數量,2歲數量,3歲數量,4歲數量,5歲數量
0,112,1,13244,16616,17363,19406,20452,21942
1,112,2,13267,16376,17210,19361,20273,21588
2,112,3,13455,16018,17101,19258,20195,21348
3,112,4,13463,15762,16936,19087,20047,21345
4,112,5,13724,15624,16809,18863,19954,21171
5,112,6,13798,15469,16831,18795,19810,21137
6,112,7,13878,15416,16762,18635,19882,21130
7,112,8,14009,15348,16735,18402,19683,21022
8,112,9,14040,15110,16797,18119,19646,20838
9,112,10,14355,14875,16741,17935,19641,20578


In [14]:
def sumChildren(month):
    idx = int(month) - 1
    sum = 0
    for i in range(6):
        sum = sum + int(population_total.at[idx,f"{i}歲數量"])
    return sum

In [15]:
population_total["幼兒數量"] = population_total["月份"].apply(
    sumChildren
)

In [16]:
population_total

,年份,月份,0歲數量,1歲數量,2歲數量,3歲數量,4歲數量,5歲數量,幼兒數量
0,112,1,13244,16616,17363,19406,20452,21942,109023
1,112,2,13267,16376,17210,19361,20273,21588,108075
2,112,3,13455,16018,17101,19258,20195,21348,107375
3,112,4,13463,15762,16936,19087,20047,21345,106640
4,112,5,13724,15624,16809,18863,19954,21171,106145
5,112,6,13798,15469,16831,18795,19810,21137,105840
6,112,7,13878,15416,16762,18635,19882,21130,105703
7,112,8,14009,15348,16735,18402,19683,21022,105199
8,112,9,14040,15110,16797,18119,19646,20838,104550
9,112,10,14355,14875,16741,17935,19641,20578,104125


In [17]:
def monthToTime(month):
    format_month = str(month).zfill(2)
    return f"2023-{format_month}-01T00:00:00+08:00"

In [27]:
result = {
    "data": [
		{
			"name": "children_count",
			"data": [
				{"y": int(population_total.at[int(month)-1, "幼兒數量"]), "x": monthToTime(month)}
				for month in reversed(population_total["月份"])
			]
		}
	]
}

In [28]:
result

{'data': [{'name': 'children_count',
   'data': [{'y': 104125, 'x': '2023-10-01T00:00:00+08:00'},
    {'y': 104550, 'x': '2023-09-01T00:00:00+08:00'},
    {'y': 105199, 'x': '2023-08-01T00:00:00+08:00'},
    {'y': 105703, 'x': '2023-07-01T00:00:00+08:00'},
    {'y': 105840, 'x': '2023-06-01T00:00:00+08:00'},
    {'y': 106145, 'x': '2023-05-01T00:00:00+08:00'},
    {'y': 106640, 'x': '2023-04-01T00:00:00+08:00'},
    {'y': 107375, 'x': '2023-03-01T00:00:00+08:00'},
    {'y': 108075, 'x': '2023-02-01T00:00:00+08:00'},
    {'y': 109023, 'x': '2023-01-01T00:00:00+08:00'}]}]}

In [1]:
with open('4002.json', "w") as json_file:
    json.dump(result, json_file, ensure_ascii=False, indent=4)

NameError: name 'json' is not defined